In [ ]:
import json
import time
import os  # Import os for file path manipulation

import requests


class Text2ImageAPI:

    def __init__(self, url, api_key, secret_key):
        self.URL = url
        self.AUTH_HEADERS = {
            'X-Key': f'Key {api_key}',
            'X-Secret': f'Secret {secret_key}',
        }

    def get_model(self):
        response = requests.get(self.URL + 'key/api/v1/models', headers=self.AUTH_HEADERS)
        data = response.json()
        return data[0]['id']

    def generate(self, prompt, model, images=1, width=1024, height=1024):
        params = {
            "type": "GENERATE",
            "numImages": images,
            "width": width,
            "height": height,
            "generateParams": {
                "query": f"{prompt}"
            }
        }

        data = {
            'model_id': (None, model),
            'params': (None, json.dumps(params), 'application/json')
        }
        response = requests.post(self.URL + 'key/api/v1/text2image/run', headers=self.AUTH_HEADERS, files=data)
        data = response.json()
        return data['uuid']

    def check_generation(self, request_id, attempts=10, delay=10):
        while attempts > 0:
            response = requests.get(self.URL + 'key/api/v1/text2image/status/' + request_id, headers=self.AUTH_HEADERS)
            data = response.json()
            if data['status'] == 'DONE':
                return data['images']

            attempts -= 1
            time.sleep(delay)

    def save_image(self, image_url, filename="output.jpg"):
        if not image_url.startswith("http"):
            image_url = f"https://{image_url}"  # Add https:// if schema is missing

        """Saves the generated image from the provided URL to a file."""
        response = requests.get(image_url, stream=True)


        if response.status_code == 200:

            # Check for a valid image extension in the URL
            if any(ext in image_url.lower() for ext in (".jpg", ".jpeg", ".png")):
                filename = os.path.splitext(image_url)[1]  # Extract extension from URL
            else:
                print("Warning: Image extension not found in URL. Using default filename.")

            with open(filename, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Image saved successfully as {filename}")
        else:
            print(f"Error downloading image: {response.status_code}")


if __name__ == '__main__':
    try:
        api = Text2ImageAPI('https://api-key.fusionbrain.ai/', '6378592DA2703EC16C70D5AC01DF61A4', '95D9F98816AB0A6432D9669E3427E0A2')
        model_id = api.get_model()
        uuid = api.generate("Sun in sky", model_id)
        generated_image_urls = api.check_generation(uuid)

        for image_url in generated_image_urls:
            api.save_image(image_url)  # Assuming this function saves the image
            print(f"Successfully generated and saved images: {image_url}")
    except Exception as e:
            print(f"An error occurred: {e}")
